In [1]:
import ray
ray.init()

@ray.remote
class NESTOrchestrator(object):
    simulator = None
    nest_model_builder = None
    nest_network = None
    
    def __init__(self):
        
        import os
        import numpy as np
        from tvb.basic.profile import TvbProfile
        TvbProfile.set_profile(TvbProfile.LIBRARY_PROFILE)

        from tvb_multiscale.tvb_nest.config import Config
        home_path = "/Users/dionperd/Software/TVB/tvb-multiscale/docs/documented_example_notebook"
        self.config = Config(output_base=os.path.join(home_path, "outputs_WilsonCowan"))

        from tvb_multiscale.tvb_nest.nest_models.builders.models.wilson_cowan import WilsonCowanBuilder
        from tvb.simulator.models.wilson_cowan_constraint import WilsonCowan
        from tvb_multiscale.core.tvb.simulator_builder import SimulatorBuilder

        model_params = {
                "r_e": np.array([0.0]),
                "r_i": np.array([0.0]),
                "k_e": np.array([1.0]),
                "k_i": np.array([1.0]),
                "tau_e": np.array([10.0]),
                "tau_i": np.array([10.0]),
                "c_ee": np.array([10.0]),
                "c_ei": np.array([6.0]),
                "c_ie": np.array([10.0]),
                "c_ii": np.array([1.0]),
                "alpha_e": np.array([1.2]),
                "alpha_i": np.array([2.0]),
                "a_e": np.array([1.0]),
                "a_i": np.array([1.0]),
                "b_e": np.array([0.0]),
                "b_i": np.array([0.0]),
                "c_e": np.array([1.0]),
                "c_i": np.array([1.0]),
                "theta_e": np.array([2.0]),
                "theta_i": np.array([3.5]),
                "P": np.array([0.5]),
                "Q": np.array([0.0])
            }

        # ----------------------1. Build a TVB simulator (model, integrator, monitors...)----------------------------------
        simulator_builder = SimulatorBuilder()
        simulator_builder.connectivity = self.config.DEFAULT_CONNECTIVITY_ZIP
        simulator_builder.model = WilsonCowan
        simulator = simulator_builder.build(**model_params)
        
        # Select the regions for the fine scale modeling with NEST spiking networks
        number_of_regions = simulator.connectivity.region_labels.shape[0]
        nest_nodes_ids = []  # the indices of fine scale regions modeled with NEST
        # We model parahippocampal cortices (left and right) with NEST
        for id in range(number_of_regions):
            if simulator.connectivity.region_labels[id].find("hippo") > 0:
                nest_nodes_ids.append(id)
        self.nest_model_builder = WilsonCowanBuilder(simulator, nest_nodes_ids, config=self.config)
        self.dt = simulator.integrator.dt
        
    def build(self):
        self.nest_network = self.nest_model_builder.build_spiking_network()
        return str(self.nest_network)
        
    def configure(self, *args, **kwargs):
        self.nest_network.configure(*args, **kwargs)
        return "NEST simulation configured!"
        
    def simulate(self, simulation_length):
        self.nest_network.Run(simulation_length * self.dt)
        return self.nest_network.nest_instance.GetKernelStatus("time")

    def do_for_nest(self, attr, *args, **kwargs):
        return getattr(self.nest_network.nest_instance, attr)(*args, **kwargs)
    
    def do_for_nest_network(self, attr, *args, **kwargs):
        return getattr(self.nest_network, attr)(*args, **kwargs)    
    

2020-10-19 19:30:35,563	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


In [2]:
nest_orchestrator = NESTOrchestrator.remote()

In [3]:
build = nest_orchestrator.build.remote()
print(ray.get(build))

(pid=89972) 2020-10-19 19:30:39,984 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - Loading a NEST instance...
(pid=89972) 2020-10-19 19:30:39,984 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - Loading a NEST instance...
(pid=89972) 2020-10-19 19:30:39,984 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - NEST_INSTALL_DIR: /Users/dionperd/Software/NEST/nest3
(pid=89972) 2020-10-19 19:30:39,984 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - NEST_INSTALL_DIR: /Users/dionperd/Software/NEST/nest3
(pid=89972) 2020-10-19 19:30:39,984 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - NEST_DATA_DIR: /Users/dionperd/Software/NEST/nest3/share/nest
(pid=89972) 2020-10-19 19:30:39,984 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - NEST_DATA_DIR: /Users/dionperd/Software/NEST/nest3/share/nest
(pid=89972) 2020-10-19 19:30:39,984 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - NEST_DOC_DIR: /Users/dionperd/Softw

(pid=89972) 
(pid=89972)               -- N E S T --
(pid=89972)   Copyright (C) 2004 The NEST Initiative
(pid=89972) 
(pid=89972)  Version: master@1e0ce51e1
(pid=89972)  Built: Oct 14 2020 22:48:45
(pid=89972) 
(pid=89972)  This program is provided AS IS and comes with
(pid=89972)  NO WARRANTY. See the file LICENSE for details.
(pid=89972) 
(pid=89972)  Problems or suggestions?
(pid=89972)    Visit https://www.nest-simulator.org
(pid=89972) 
(pid=89972)  Type 'nest.help()' to find out more about NEST.
(pid=89972) 
(pid=89972) 2020-10-19 19:30:40,826 - INFO - tvb_multiscale.core.spiking_models.devices - <class 'tvb_multiscale.core.spiking_models.devices.DeviceSet'> of model spike_recorder for E created!
(pid=89972) 2020-10-19 19:30:40,826 - INFO - tvb_multiscale.core.spiking_models.devices - <class 'tvb_multiscale.core.spiking_models.devices.DeviceSet'> of model spike_recorder for E created!
(pid=89972) 2020-10-19 19:30:40,889 - INFO - tvb_multiscale.core.spiking_models.devices - <clas

In [4]:
configure = nest_orchestrator.configure.remote()
print(ray.get(configure))

NEST simulation configured!


In [5]:
simulate = nest_orchestrator.simulate.remote(1000)
print(ray.get(simulate))

100.0


In [6]:
time = nest_orchestrator.do_for_nest.remote("GetKernelStatus", "time")
print(ray.get(time))

100.0


In [7]:
print_str = nest_orchestrator.do_for_nest_network.remote("print_str", False)
print(ray.get(print_str))


----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
NESTNetwork:

----------------------------------------------------------------------------------------------------

SpikingBrain - Regions: ['parahippocampal_L', 'parahippocampal_R']
Regions' nodes:

----------------------------------------------------------------------------------------------------
NESTRegionNode - Label: parahippocampal_L
Populations ['E', 'I']:

----------------------------------------------------------------------------------------------------

NESTPopulation - Label: E 
100 neurons: 
100 neurons: 1...100
parameters: {'archiver_length': [[0]], 'beta_Ca': array([0.]), 'C_m': array([250.]), 'Ca': array([0.]), 'dg_ex': array([54.7, 10.8, 10.6,  9.6, 17. , 34.6, 23.6, 

In [8]:

def start_NESTOrchestrator_server():
    from xmlrpc.server import SimpleXMLRPCServer
    from xmlrpc.server import SimpleXMLRPCRequestHandler

    # Restrict to a particular path.
    class RequestHandler(SimpleXMLRPCRequestHandler):
        rpc_paths = ('/RPC2',)

    # Create server
    with SimpleXMLRPCServer(('localhost', 8000),
                            requestHandler=RequestHandler) as server:
        server.register_introspection_functions()
        
        class NESTOrchestrator():

            nest_network = None
            
            def build_nest_network(self):

                import os
                import numpy as np
                from tvb.basic.profile import TvbProfile
                TvbProfile.set_profile(TvbProfile.LIBRARY_PROFILE)

                from tvb_multiscale.tvb_nest.config import Config
                home_path = "/Users/dionperd/Software/TVB/tvb-multiscale/docs/documented_example_notebook"
                self.config = Config(output_base=os.path.join(home_path, "outputs_WilsonCowan"))

                from tvb_multiscale.tvb_nest.nest_models.builders.models.wilson_cowan import WilsonCowanBuilder
                from tvb.simulator.models.wilson_cowan_constraint import WilsonCowan
                from tvb_multiscale.core.tvb.simulator_builder import SimulatorBuilder

                model_params = {
                        "r_e": np.array([0.0]),
                        "r_i": np.array([0.0]),
                        "k_e": np.array([1.0]),
                        "k_i": np.array([1.0]),
                        "tau_e": np.array([10.0]),
                        "tau_i": np.array([10.0]),
                        "c_ee": np.array([10.0]),
                        "c_ei": np.array([6.0]),
                        "c_ie": np.array([10.0]),
                        "c_ii": np.array([1.0]),
                        "alpha_e": np.array([1.2]),
                        "alpha_i": np.array([2.0]),
                        "a_e": np.array([1.0]),
                        "a_i": np.array([1.0]),
                        "b_e": np.array([0.0]),
                        "b_i": np.array([0.0]),
                        "c_e": np.array([1.0]),
                        "c_i": np.array([1.0]),
                        "theta_e": np.array([2.0]),
                        "theta_i": np.array([3.5]),
                        "P": np.array([0.5]),
                        "Q": np.array([0.0])
                    }

                # ----------------------1. Build a TVB simulator (model, integrator, monitors...)----------------------------------
                simulator_builder = SimulatorBuilder()
                simulator_builder.connectivity = self.config.DEFAULT_CONNECTIVITY_ZIP
                simulator_builder.model = WilsonCowan
                simulator = simulator_builder.build(**model_params)

                # Select the regions for the fine scale modeling with NEST spiking networks
                number_of_regions = simulator.connectivity.region_labels.shape[0]
                nest_nodes_ids = []  # the indices of fine scale regions modeled with NEST
                # We model parahippocampal cortices (left and right) with NEST
                for id in range(number_of_regions):
                    if simulator.connectivity.region_labels[id].find("hippo") > 0:
                        nest_nodes_ids.append(id)
                nest_model_builder = WilsonCowanBuilder(simulator, nest_nodes_ids, config=self.config)
                self.dt = simulator.integrator.dt
                self.nest_network = nest_model_builder.build_spiking_network()
                return str(self.nest_network)

            def configure(self, *args, **kwargs):
                self.nest_network.configure(*args, **kwargs)
                return "NEST simulation configured!"

            def simulate(self, simulation_length):
                self.nest_network.Run(simulation_length * self.dt)
                return self.nest_network.nest_instance.GetKernelStatus("time")

            def do_for_nest(self, attr, *args, **kwargs):
                return getattr(self.nest_network.nest_instance, attr)(*args, **kwargs)
            
            def do_for_nest_network(self, attr, *args, **kwargs):
                return str(getattr(self.nest_network, attr)(*args, **kwargs))

        server.register_instance(NESTOrchestrator())

        # Run the server's main loop
        server.serve_forever()


In [9]:
from multiprocessing import Process

p = Process(target=start_NESTOrchestrator_server)
p.start()
# p.join()

2020-10-19 19:30:45,032 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - Loading a NEST instance...
2020-10-19 19:30:45,032 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - Loading a NEST instance...
2020-10-19 19:30:45,038 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - NEST_INSTALL_DIR: /Users/dionperd/Software/NEST/nest3
2020-10-19 19:30:45,038 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - NEST_INSTALL_DIR: /Users/dionperd/Software/NEST/nest3
2020-10-19 19:30:45,043 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - NEST_DATA_DIR: /Users/dionperd/Software/NEST/nest3/share/nest
2020-10-19 19:30:45,043 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - NEST_DATA_DIR: /Users/dionperd/Software/NEST/nest3/share/nest
2020-10-19 19:30:45,049 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - NEST_DOC_DIR: /Users/dionperd/Software/NEST/nest3/share/doc/nest
2020-10-19 19:30:45,049 - INFO - tvb_multiscale.tvb_ne

2020-10-19 19:30:45,082 - INFO - tvb_multiscale.tvb_nest.nest_models.builders.base - system path: ['/Users/dionperd/Software/NEST/nest3/lib/python3.7/site-packages', '/Users/dionperd/opt/anaconda3/lib/python3.7/site-packages/ray/thirdparty_files', '/Users/dionperd', '/Users/dionperd/opt/anaconda3/lib/python3.7/site-packages', '/Users/dionperd', '/Users/dionperd/opt/anaconda3/lib/python3.7/site-packages', '/Users/dionperd/opt/anaconda3/lib/python3.7/site-packages/ray/pickle5_files', '/Users/dionperd', '', '/Users/dionperd/Software/NEST/nest3/lib/python3.7/site-packages', '/Users/dionperd/opt/anaconda3/lib/python37.zip', '/Users/dionperd/opt/anaconda3/lib/python3.7', '/Users/dionperd/opt/anaconda3/lib/python3.7/lib-dynload', '/Users/dionperd/opt/anaconda3/lib/python3.7/site-packages', '/Users/dionperd/opt/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/dionperd/Software/TVB/tvb-data', '/Users/dionperd/opt/anaconda3/lib/python3.7/site-packages/xarray-0.15.0-py3.7.egg', '/Users/dionp

127.0.0.1 - - [19/Oct/2020 19:30:46] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2020 19:30:46] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2020 19:30:46] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2020 19:30:46] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2020 19:30:47] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2020 19:30:47] "POST /RPC2 HTTP/1.1" 200 -


In [10]:
import xmlrpc.client

s = xmlrpc.client.ServerProxy('http://localhost:8000')

In [11]:
print(s.build_nest_network())


----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
NESTNetwork:

----------------------------------------------------------------------------------------------------

SpikingBrain - Regions: ['parahippocampal_L', 'parahippocampal_R']
Regions' nodes:

----------------------------------------------------------------------------------------------------
NESTRegionNode - Label: parahippocampal_L
Populations ['E', 'I']:

----------------------------------------------------------------------------------------------------

NESTPopulation - Label: E 
100 neurons: 
100 neurons: 1...100
parameters: {'archiver_length': [[0]], 'beta_Ca': array([0.]), 'C_m': array([250.]), 'Ca': array([0.]), 'dg_ex': array([0.]), 'dg_in': array([0.]), 'E_L': array([

In [12]:
print(s.configure())

NEST simulation configured!


In [13]:
print(s.simulate(1000))

100.0


In [14]:
print(s.do_for_nest("GetKernelStatus", "time"))

100.0


In [15]:
print(s.simulate(1000))

200.0


In [16]:
print(s.do_for_nest_network("__str__"))


----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
NESTNetwork:

----------------------------------------------------------------------------------------------------

SpikingBrain - Regions: ['parahippocampal_L', 'parahippocampal_R']
Regions' nodes:

----------------------------------------------------------------------------------------------------
NESTRegionNode - Label: parahippocampal_L
Populations ['E', 'I']:

----------------------------------------------------------------------------------------------------

NESTPopulation - Label: E 
100 neurons: 
100 neurons: 1...100
parameters: {'archiver_length': [[0]], 'beta_Ca': array([0.]), 'C_m': array([250.]), 'Ca': array([0.]), 'dg_ex': array([18.5,  2.9, 24. , 14.7,  6.4, 16.5, 24.6, 